# Building the Invoke Agent Skill
This notebook demonstrates how to use the Cortex Python SDK to build a skill which will invoke an Agent. It will take two input numbers, call upon the sum-returner agent, and return their sum. 

In [ ]:
# Common Setup
%run setup.ipynb
cortex = Cortex.client()
builder = cortex.builder()

## Define the invoke_agent function

This function is broken up into two functions for readbility, invoke_agent and call_agent. invoke_agent takes two numbers as input, and passes them to call_agent. call_agent creates a JSON payload from the numbers, and then initializes the Cortex client with the users token in order to invoke the sum-returner agent. It passes the payload to this agent and gets the result, returning it to the original function: invoke_agent. invoke_agent finally returns the result.

In [ ]:
%%cortex_action --name 'class/invoke_agent-<your_initials>' --function invoke_agent
from cortex import Message

def invoke_agent(params):
    msg = Message(params)
    num1 = msg.payload.get('num1')
    num2 = msg.payload.get('num2')
    
    result = call_agent(num1, num2, msg.apiEndpoint, msg.token)
    
    return Message.with_payload({'Result': result}).to_params()

def call_agent(num1, num2, url, token):
    from cortex import Cortex
    
    cortex = Cortex.client(api_endpoint = url, token=token)
    agent = cortex.agent("default/calculate-sum-<your_initials>")
    
    rs = agent.invoke_service("calculate-sum-<your_initials>", Message.with_payload({'num1': 10, 'num2' :15}))
    res = rs.payload.get('Sum')
    
    return res

Build and Deploy Action

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
action = cortex.action('class/invoke_agent-<your_initials>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _eq_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'num1':10, 'num2':22}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Invoke Agent" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/invoke-agent-<your_initials>').title('Invoke Agent-<your_initials>').description('Skill which invokes the Sum Returner agent. This agent will take the two numbers from the skill, and return the sum as a json object.')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('numbers').title('Input Numbers').parameter(name='num1', type='number').parameter(name='num2',type='number').all_routing(action, 'result').build()


In the previous step, we referenced an Output called **result**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('result').title('Result').parameter(name='Result', type='number').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='numbers', message=Message.with_payload({'num1': 12500, 'num2':3}))
rs.payload

In [ ]:
def invoke_again():
    
    agent = cortex.agent("default/calculate-sum-<your_initials>")
    rs= agent.invoke_service("calculate-sum-<your_initials>", Message.with_payload({'num1' : 10, 'num2' : 15}))
    print(rs.payload.get('Sum'))
          
invoke_again()